# Pitfall 02: CUPED 适用条件误用

## 🎯 学习目标

完成本 notebook 后，你将能够：

1. **识别** CUPED 的适用条件和常见误用场景
2. **诊断** CUPED 失效的原因
3. **掌握** 正确使用 CUPED 的最佳实践
4. **应对** 面试中关于 CUPED 的深度追问

---

## 📖 背景故事

> **场景：实验平台的困惑**
>
> 你是某大厂实验平台的数据科学家。最近产品团队抱怨："我们用了 CUPED，但方差缩减效果不明显，有时甚至变差了！"
>
> 经过排查，你发现团队在以下场景错误使用了 CUPED：
> - 新用户实验（没有历史数据）
> - 协变量与结果相关性很低
> - 处理效应影响了协变量
> - 样本量太小
>
> **本 notebook 将深入分析这些陷阱，帮你彻底理解 CUPED。**

---

## Part 1: CUPED 原理快速回顾

### 1.1 核心思想

CUPED (Controlled-experiment Using Pre-Experiment Data) 的核心思想是：

**利用实验前的协变量 X 与结果 Y 的相关性，减少结果变量的方差。**

### 1.2 数学推导

设：
- $Y$ 为结果变量
- $X$ 为实验前协变量（如历史指标）

CUPED 调整后的结果：
$$Y_{adj} = Y - \theta (X - \bar{X})$$

其中 $\theta = \frac{Cov(Y, X)}{Var(X)}$ 是最小化 $Var(Y_{adj})$ 的最优系数。

### 1.3 方差缩减

$$Var(Y_{adj}) = Var(Y)(1 - \rho^2_{XY})$$

其中 $\rho_{XY}$ 是 $X$ 和 $Y$ 的相关系数。

**关键洞见**：方差缩减程度取决于相关性 $\rho^2$！

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial Unicode MS', 'SimHei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 环境准备完成")

In [ ]:
def cuped_adjust(Y, X):
    """
    标准 CUPED 调整
    
    Args:
        Y: 结果变量
        X: 协变量（实验前数据）
        
    Returns:
        Y_adj: 调整后的结果
        theta: 调整系数
        rho: 相关系数
    """
    # 计算 theta
    cov_xy = np.cov(Y, X)[0, 1]
    var_x = np.var(X, ddof=1)
    theta = cov_xy / var_x if var_x > 0 else 0
    
    # 计算相关系数
    rho = np.corrcoef(Y, X)[0, 1]
    
    # CUPED 调整
    Y_adj = Y - theta * (X - np.mean(X))
    
    return Y_adj, theta, rho

def run_ab_test(Y_control, Y_treatment, alpha=0.05):
    """
    运行 A/B 测试并返回结果
    """
    n_c, n_t = len(Y_control), len(Y_treatment)
    
    # 均值差
    diff = np.mean(Y_treatment) - np.mean(Y_control)
    
    # 标准误
    se = np.sqrt(np.var(Y_control, ddof=1)/n_c + np.var(Y_treatment, ddof=1)/n_t)
    
    # t 统计量和 p 值
    t_stat = diff / se
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=n_c + n_t - 2))
    
    # 置信区间
    ci = stats.t.interval(1-alpha, df=n_c+n_t-2, loc=diff, scale=se)
    
    return {
        'diff': diff,
        'se': se,
        't_stat': t_stat,
        'p_value': p_value,
        'ci': ci,
        'significant': p_value < alpha
    }

print("✅ 工具函数定义完成")

---

## Part 2: 失败模式 1 - 协变量与结果相关性低

### 2.1 问题描述

**常见误区**："只要有历史数据，用 CUPED 就一定能减少方差"

**真相**：如果 $\rho_{XY}$ 很低，方差缩减效果微乎其微，甚至可能因为估计误差反而变差！

In [ ]:
def simulate_low_correlation_cuped(n_samples=1000, true_effect=2.0, rho=0.1, n_simulations=500):
    """
    模拟低相关性场景下的 CUPED 效果
    """
    results_raw = []
    results_cuped = []
    
    for _ in range(n_simulations):
        # 生成相关的 X 和 Y
        # Y = rho * X + sqrt(1-rho^2) * noise + treatment_effect
        X = np.random.randn(n_samples)
        noise = np.random.randn(n_samples)
        
        # 控制组
        Y_c = rho * X[:n_samples//2] + np.sqrt(1-rho**2) * noise[:n_samples//2]
        X_c = X[:n_samples//2]
        
        # 实验组
        Y_t = rho * X[n_samples//2:] + np.sqrt(1-rho**2) * noise[n_samples//2:] + true_effect
        X_t = X[n_samples//2:]
        
        # 原始 A/B 测试
        result_raw = run_ab_test(Y_c, Y_t)
        results_raw.append(result_raw)
        
        # CUPED 调整
        Y_c_adj, _, _ = cuped_adjust(Y_c, X_c)
        Y_t_adj, _, _ = cuped_adjust(Y_t, X_t)
        
        result_cuped = run_ab_test(Y_c_adj, Y_t_adj)
        results_cuped.append(result_cuped)
    
    return results_raw, results_cuped

# 对比不同相关性
correlations = [0.1, 0.3, 0.5, 0.7, 0.9]
variance_reduction = []
power_improvement = []

print("模拟不同相关性下的 CUPED 效果...\n")

for rho in correlations:
    results_raw, results_cuped = simulate_low_correlation_cuped(n_samples=500, rho=rho, n_simulations=300)
    
    # 计算方差缩减
    var_raw = np.mean([r['se']**2 for r in results_raw])
    var_cuped = np.mean([r['se']**2 for r in results_cuped])
    var_reduction = 1 - var_cuped / var_raw
    variance_reduction.append(var_reduction)
    
    # 计算 Power 提升
    power_raw = np.mean([r['significant'] for r in results_raw])
    power_cuped = np.mean([r['significant'] for r in results_cuped])
    power_improvement.append(power_cuped - power_raw)
    
    print(f"相关性 ρ = {rho:.1f}:")
    print(f"  理论方差缩减: {rho**2:.1%}")
    print(f"  实际方差缩减: {var_reduction:.1%}")
    print(f"  Power 提升: {power_cuped - power_raw:.1%}")
    print()

In [ ]:
# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 方差缩减
ax1 = axes[0]
theoretical = [rho**2 for rho in correlations]
x = np.arange(len(correlations))
width = 0.35

ax1.bar(x - width/2, theoretical, width, label='理论值', color='#3498db', alpha=0.8)
ax1.bar(x + width/2, variance_reduction, width, label='实际值', color='#e74c3c', alpha=0.8)
ax1.set_xticks(x)
ax1.set_xticklabels([f'ρ={r}' for r in correlations])
ax1.set_ylabel('方差缩减比例')
ax1.set_title('CUPED 方差缩减：理论 vs 实际')
ax1.legend()
ax1.axhline(y=0.1, color='gray', linestyle='--', alpha=0.5, label='10% 阈值')

# Power 提升
ax2 = axes[1]
colors = ['#e74c3c' if p < 0.02 else '#2ecc71' for p in power_improvement]
ax2.bar(correlations, power_improvement, color=colors, alpha=0.8, width=0.15)
ax2.set_xlabel('相关系数 ρ')
ax2.set_ylabel('Power 提升')
ax2.set_title('CUPED 带来的 Power 提升')
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 关键发现：")
print(f"   当 ρ < 0.3 时，CUPED 方差缩减 < 10%，效果有限")
print(f"   实际方差缩减可能略低于理论值（因为 θ 需要估计）")

### 2.2 诊断方法

**在应用 CUPED 前，先检查相关性！**

In [ ]:
def cuped_diagnostic(Y, X, significance_level=0.05):
    """
    CUPED 诊断函数
    判断是否适合使用 CUPED
    """
    # 计算相关性
    rho, p_value = stats.pearsonr(Y, X)
    
    # 理论方差缩减
    theoretical_reduction = rho ** 2
    
    # 判断
    recommendations = []
    
    if abs(rho) < 0.3:
        recommendations.append("⚠️ 相关性较低 (|ρ| < 0.3)，CUPED 效果有限")
    elif abs(rho) < 0.5:
        recommendations.append("⚡ 相关性中等 (0.3 ≤ |ρ| < 0.5)，CUPED 有一定效果")
    else:
        recommendations.append("✅ 相关性较强 (|ρ| ≥ 0.5)，推荐使用 CUPED")
    
    if p_value > significance_level:
        recommendations.append("⚠️ 相关性不显著，CUPED 可能引入噪声")
    
    print("=" * 50)
    print("CUPED 诊断报告")
    print("=" * 50)
    print(f"\n相关系数: ρ = {rho:.4f}")
    print(f"相关性 p-value: {p_value:.4f}")
    print(f"理论方差缩减: {theoretical_reduction:.2%}")
    print(f"\n建议：")
    for rec in recommendations:
        print(f"  {rec}")
    
    return {
        'rho': rho,
        'p_value': p_value,
        'theoretical_reduction': theoretical_reduction,
        'recommendations': recommendations
    }

# 示例
# 低相关性场景
X_low = np.random.randn(1000)
Y_low = 0.1 * X_low + np.random.randn(1000)

print("场景 1: 低相关性")
cuped_diagnostic(Y_low, X_low)

print("\n")

# 高相关性场景
X_high = np.random.randn(1000)
Y_high = 0.8 * X_high + 0.6 * np.random.randn(1000)

print("场景 2: 高相关性")
cuped_diagnostic(Y_high, X_high)

---

## Part 3: 失败模式 2 - 新用户没有历史数据

### 3.1 问题描述

**场景**：对新用户进行实验，但新用户没有历史行为数据。

**常见错误做法**：
1. 用 0 填充缺失值
2. 用全体均值填充
3. 强行使用注册后的数据作为"历史"

In [ ]:
def simulate_new_user_cuped(n_samples=2000, new_user_ratio=0.3, true_effect=2.0):
    """
    模拟新用户场景下的 CUPED 问题
    """
    n_new = int(n_samples * new_user_ratio)
    n_old = n_samples - n_new
    
    # 老用户：有历史数据，相关性高
    X_old = np.random.randn(n_old) * 10 + 50  # 历史 GMV
    noise_old = np.random.randn(n_old) * 5
    Y_old = 0.7 * X_old + noise_old + 20  # 当期 GMV
    
    # 新用户：无历史数据
    X_new_missing = np.full(n_new, np.nan)
    noise_new = np.random.randn(n_new) * 8  # 新用户方差更大
    Y_new = 30 + noise_new  # 新用户基础值不同
    
    # 合并
    X_all = np.concatenate([X_old, X_new_missing])
    Y_all = np.concatenate([Y_old, Y_new])
    is_new = np.concatenate([np.zeros(n_old), np.ones(n_new)])
    
    # 随机分组
    perm = np.random.permutation(n_samples)
    X_all, Y_all, is_new = X_all[perm], Y_all[perm], is_new[perm]
    
    # 分为控制组和实验组
    n_half = n_samples // 2
    X_c, Y_c, is_new_c = X_all[:n_half], Y_all[:n_half], is_new[:n_half]
    X_t, Y_t, is_new_t = X_all[n_half:], Y_all[n_half:] + true_effect, is_new[n_half:]
    
    return X_c, Y_c, is_new_c, X_t, Y_t, is_new_t

# 生成数据
X_c, Y_c, is_new_c, X_t, Y_t, is_new_t = simulate_new_user_cuped(n_samples=2000, new_user_ratio=0.4)

print("数据概况：")
print(f"  控制组: {len(Y_c)} 样本, 新用户 {is_new_c.sum():.0f} ({is_new_c.mean():.1%})")
print(f"  实验组: {len(Y_t)} 样本, 新用户 {is_new_t.sum():.0f} ({is_new_t.mean():.1%})")
print(f"  缺失协变量: {np.isnan(X_c).sum() + np.isnan(X_t).sum()} 个")

In [ ]:
# 对比不同处理缺失值的方法

def compare_missing_strategies(X_c, Y_c, X_t, Y_t):
    """
    对比不同的缺失值处理策略
    """
    results = {}
    
    # 策略 0: 不使用 CUPED（基准）
    result_raw = run_ab_test(Y_c, Y_t)
    results['不使用 CUPED'] = result_raw
    
    # 策略 1: 用 0 填充
    X_c_zero = np.nan_to_num(X_c, nan=0)
    X_t_zero = np.nan_to_num(X_t, nan=0)
    Y_c_adj, _, _ = cuped_adjust(Y_c, X_c_zero)
    Y_t_adj, _, _ = cuped_adjust(Y_t, X_t_zero)
    results['填充 0'] = run_ab_test(Y_c_adj, Y_t_adj)
    
    # 策略 2: 用均值填充
    X_c_mean = np.nan_to_num(X_c, nan=np.nanmean(X_c))
    X_t_mean = np.nan_to_num(X_t, nan=np.nanmean(X_t))
    Y_c_adj, _, _ = cuped_adjust(Y_c, X_c_mean)
    Y_t_adj, _, _ = cuped_adjust(Y_t, X_t_mean)
    results['填充均值'] = run_ab_test(Y_c_adj, Y_t_adj)
    
    # 策略 3: 只对有历史数据的用户使用 CUPED
    mask_c = ~np.isnan(X_c)
    mask_t = ~np.isnan(X_t)
    
    if mask_c.sum() > 0 and mask_t.sum() > 0:
        Y_c_old_adj, theta_c, _ = cuped_adjust(Y_c[mask_c], X_c[mask_c])
        Y_t_old_adj, theta_t, _ = cuped_adjust(Y_t[mask_t], X_t[mask_t])
        
        # 新用户不调整
        Y_c_combined = np.concatenate([Y_c_old_adj, Y_c[~mask_c]])
        Y_t_combined = np.concatenate([Y_t_old_adj, Y_t[~mask_t]])
        results['分层 CUPED'] = run_ab_test(Y_c_combined, Y_t_combined)
    
    # 策略 4: 只分析老用户（可能引入选择偏差）
    if mask_c.sum() > 0 and mask_t.sum() > 0:
        Y_c_adj, _, _ = cuped_adjust(Y_c[mask_c], X_c[mask_c])
        Y_t_adj, _, _ = cuped_adjust(Y_t[mask_t], X_t[mask_t])
        results['仅老用户+CUPED'] = run_ab_test(Y_c_adj, Y_t_adj)
    
    return results

strategies_results = compare_missing_strategies(X_c, Y_c, X_t, Y_t)

print("=" * 70)
print("不同缺失值处理策略对比")
print("=" * 70)
print(f"\n真实效应 = 2.0")
print(f"\n{'策略':<20} {'估计效应':<12} {'标准误':<12} {'p-value':<12}")
print("-" * 56)

for strategy, result in strategies_results.items():
    print(f"{strategy:<20} {result['diff']:<12.3f} {result['se']:<12.3f} {result['p_value']:<12.4f}")

In [ ]:
# 可视化策略效果
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

strategies = list(strategies_results.keys())
se_values = [strategies_results[s]['se'] for s in strategies]
diff_values = [strategies_results[s]['diff'] for s in strategies]

# 标准误对比
ax1 = axes[0]
colors = ['#3498db' if s == '分层 CUPED' else '#95a5a6' for s in strategies]
bars = ax1.bar(range(len(strategies)), se_values, color=colors)
ax1.set_xticks(range(len(strategies)))
ax1.set_xticklabels(strategies, rotation=45, ha='right')
ax1.set_ylabel('标准误')
ax1.set_title('不同策略的标准误')
ax1.axhline(y=se_values[0], color='red', linestyle='--', alpha=0.5, label='基准')

# 效应估计偏差
ax2 = axes[1]
true_effect = 2.0
bias = [d - true_effect for d in diff_values]
colors = ['#2ecc71' if abs(b) < 0.5 else '#e74c3c' for b in bias]
ax2.bar(range(len(strategies)), bias, color=colors)
ax2.set_xticks(range(len(strategies)))
ax2.set_xticklabels(strategies, rotation=45, ha='right')
ax2.set_ylabel('估计偏差 (估计值 - 真实值)')
ax2.set_title('不同策略的估计偏差')
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 关键发现：")
print("  1. 用 0 填充会引入系统偏差（新用户的 0 不是真实的历史值）")
print("  2. 用均值填充会削弱 CUPED 效果（降低了相关性）")
print("  3. 分层 CUPED（老用户用 CUPED，新用户不用）是最佳策略")
print("  4. 仅分析老用户可能引入选择偏差")

### 3.2 正确做法：分层 CUPED

**最佳实践**：
1. 将用户分为"有历史数据"和"无历史数据"两层
2. 对有历史数据的用户使用 CUPED
3. 对无历史数据的用户使用原始值
4. 合并两层的结果

In [ ]:
def stratified_cuped(Y_c, X_c, Y_t, X_t):
    """
    分层 CUPED 的正确实现
    """
    # 识别有/无历史数据的用户
    mask_c = ~np.isnan(X_c)
    mask_t = ~np.isnan(X_t)
    
    # 有历史数据的用户：使用 CUPED
    if mask_c.sum() > 0:
        Y_c_old_adj, theta_c, rho_c = cuped_adjust(Y_c[mask_c], X_c[mask_c])
        print(f"老用户 CUPED 参数: θ={theta_c:.3f}, ρ={rho_c:.3f}")
    else:
        Y_c_old_adj = np.array([])
    
    if mask_t.sum() > 0:
        Y_t_old_adj, theta_t, rho_t = cuped_adjust(Y_t[mask_t], X_t[mask_t])
    else:
        Y_t_old_adj = np.array([])
    
    # 无历史数据的用户：使用原始值
    Y_c_new = Y_c[~mask_c]
    Y_t_new = Y_t[~mask_t]
    
    # 合并
    Y_c_final = np.concatenate([Y_c_old_adj, Y_c_new])
    Y_t_final = np.concatenate([Y_t_old_adj, Y_t_new])
    
    print(f"\n分层统计:")
    print(f"  控制组: 老用户 {mask_c.sum()}, 新用户 {(~mask_c).sum()}")
    print(f"  实验组: 老用户 {mask_t.sum()}, 新用户 {(~mask_t).sum()}")
    
    return Y_c_final, Y_t_final

# 应用分层 CUPED
print("分层 CUPED 应用:")
print("=" * 50)
Y_c_final, Y_t_final = stratified_cuped(Y_c, X_c, Y_t, X_t)

result = run_ab_test(Y_c_final, Y_t_final)
print(f"\n效应估计: {result['diff']:.3f} (真实值: 2.0)")
print(f"标准误: {result['se']:.3f}")
print(f"p-value: {result['p_value']:.4f}")

---

## Part 4: 失败模式 3 - 处理效应影响协变量

### 4.1 问题描述

**CUPED 的核心假设**：协变量 $X$ 在实验前测量，不受处理的影响。

**违反场景**：
- 使用实验期间的数据作为协变量
- 协变量是结果变量的一部分
- 协变量受到处理的间接影响

In [ ]:
def simulate_confounded_cuped(n_samples=2000, true_effect=2.0, n_simulations=300):
    """
    模拟协变量受处理影响的场景
    
    错误场景：使用实验第一周的数据作为"协变量"来预测第二周
    问题：第一周的数据已经受到处理的影响！
    """
    results_correct = []  # 使用实验前数据
    results_wrong = []    # 使用实验期间数据（错误）
    
    for _ in range(n_simulations):
        # 实验前数据（未受处理影响）
        X_pre = np.random.randn(n_samples) * 10 + 50
        
        # 分组
        n_half = n_samples // 2
        X_pre_c, X_pre_t = X_pre[:n_half], X_pre[n_half:]
        
        # 第一周结果（已受处理影响）
        Y_week1_c = 0.7 * X_pre_c + np.random.randn(n_half) * 5 + 20
        Y_week1_t = 0.7 * X_pre_t + np.random.randn(n_half) * 5 + 20 + true_effect * 0.5  # 受处理影响
        
        # 第二周结果（目标变量）
        Y_week2_c = 0.6 * X_pre_c + 0.3 * Y_week1_c + np.random.randn(n_half) * 4 + 10
        Y_week2_t = 0.6 * X_pre_t + 0.3 * Y_week1_t + np.random.randn(n_half) * 4 + 10 + true_effect  # 主效应
        
        # 正确做法：使用实验前数据
        Y_c_adj, _, _ = cuped_adjust(Y_week2_c, X_pre_c)
        Y_t_adj, _, _ = cuped_adjust(Y_week2_t, X_pre_t)
        results_correct.append(run_ab_test(Y_c_adj, Y_t_adj))
        
        # 错误做法：使用第一周数据（已受处理影响）
        Y_c_adj_wrong, _, _ = cuped_adjust(Y_week2_c, Y_week1_c)
        Y_t_adj_wrong, _, _ = cuped_adjust(Y_week2_t, Y_week1_t)
        results_wrong.append(run_ab_test(Y_c_adj_wrong, Y_t_adj_wrong))
    
    return results_correct, results_wrong

# 运行模拟
print("模拟协变量受处理影响的场景...\n")
results_correct, results_wrong = simulate_confounded_cuped(n_simulations=500)

# 分析结果
true_effect = 2.0

estimates_correct = [r['diff'] for r in results_correct]
estimates_wrong = [r['diff'] for r in results_wrong]

print("=" * 60)
print("协变量选择的影响")
print("=" * 60)
print(f"\n真实效应: {true_effect}")
print(f"\n{'方法':<25} {'平均估计':<12} {'偏差':<12} {'标准差':<12}")
print("-" * 60)
print(f"{'正确: 使用实验前数据':<25} {np.mean(estimates_correct):<12.3f} {np.mean(estimates_correct)-true_effect:<12.3f} {np.std(estimates_correct):<12.3f}")
print(f"{'错误: 使用第一周数据':<25} {np.mean(estimates_wrong):<12.3f} {np.mean(estimates_wrong)-true_effect:<12.3f} {np.std(estimates_wrong):<12.3f}")

In [ ]:
# 可视化估计分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 估计分布对比
ax1 = axes[0]
ax1.hist(estimates_correct, bins=30, alpha=0.6, label='正确: 实验前数据', color='#2ecc71')
ax1.hist(estimates_wrong, bins=30, alpha=0.6, label='错误: 第一周数据', color='#e74c3c')
ax1.axvline(x=true_effect, color='black', linestyle='--', linewidth=2, label=f'真实效应={true_effect}')
ax1.set_xlabel('效应估计')
ax1.set_ylabel('频数')
ax1.set_title('效应估计分布对比')
ax1.legend()

# 偏差分析
ax2 = axes[1]
bias_correct = np.array(estimates_correct) - true_effect
bias_wrong = np.array(estimates_wrong) - true_effect

data_to_plot = [bias_correct, bias_wrong]
bp = ax2.boxplot(data_to_plot, labels=['正确', '错误'], patch_artist=True)
bp['boxes'][0].set_facecolor('#2ecc71')
bp['boxes'][1].set_facecolor('#e74c3c')
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax2.set_ylabel('估计偏差')
ax2.set_title('估计偏差分布')

plt.tight_layout()
plt.show()

print("\n💡 关键发现：")
print(f"   使用受处理影响的协变量会导致估计偏差")
print(f"   偏差方向：通常低估处理效应")
print(f"   原因：协变量"吸收"了部分处理效应")

---

## Part 5: 失败模式 4 - 样本量过小

### 5.1 问题描述

CUPED 需要估计 θ = Cov(Y,X)/Var(X)，当样本量小时：
- θ 的估计方差大
- 可能引入额外噪声
- 方差缩减效果不稳定

In [ ]:
def simulate_small_sample_cuped(sample_sizes=[50, 100, 200, 500, 1000, 2000], 
                                  rho=0.6, true_effect=2.0, n_simulations=500):
    """
    模拟不同样本量下的 CUPED 效果
    """
    results = {}
    
    for n in sample_sizes:
        var_reductions = []
        biases = []
        
        for _ in range(n_simulations):
            # 生成数据
            X = np.random.randn(n)
            noise = np.random.randn(n)
            
            n_half = n // 2
            Y_c = rho * X[:n_half] + np.sqrt(1-rho**2) * noise[:n_half]
            Y_t = rho * X[n_half:] + np.sqrt(1-rho**2) * noise[n_half:] + true_effect
            X_c, X_t = X[:n_half], X[n_half:]
            
            # 原始方差
            var_raw = np.var(Y_c, ddof=1) + np.var(Y_t, ddof=1)
            
            # CUPED
            Y_c_adj, _, _ = cuped_adjust(Y_c, X_c)
            Y_t_adj, _, _ = cuped_adjust(Y_t, X_t)
            var_cuped = np.var(Y_c_adj, ddof=1) + np.var(Y_t_adj, ddof=1)
            
            var_reductions.append(1 - var_cuped / var_raw)
            
            # 估计偏差
            estimate = np.mean(Y_t_adj) - np.mean(Y_c_adj)
            biases.append(estimate - true_effect)
        
        results[n] = {
            'var_reduction_mean': np.mean(var_reductions),
            'var_reduction_std': np.std(var_reductions),
            'bias_mean': np.mean(biases),
            'bias_std': np.std(biases)
        }
    
    return results

# 运行模拟
print("模拟不同样本量的 CUPED 效果...\n")
sample_size_results = simulate_small_sample_cuped()

print("=" * 70)
print(f"不同样本量下的 CUPED 效果 (理论方差缩减 = {0.6**2:.1%})")
print("=" * 70)
print(f"\n{'样本量':<10} {'方差缩减均值':<15} {'方差缩减标准差':<15} {'估计偏差均值':<15}")
print("-" * 60)

for n, r in sample_size_results.items():
    print(f"{n:<10} {r['var_reduction_mean']:<15.2%} {r['var_reduction_std']:<15.2%} {r['bias_mean']:<15.4f}")

In [ ]:
# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sample_sizes = list(sample_size_results.keys())
var_reduction_means = [sample_size_results[n]['var_reduction_mean'] for n in sample_sizes]
var_reduction_stds = [sample_size_results[n]['var_reduction_std'] for n in sample_sizes]

# 方差缩减
ax1 = axes[0]
ax1.errorbar(sample_sizes, var_reduction_means, yerr=var_reduction_stds, 
            fmt='o-', capsize=5, capthick=2, color='#3498db')
ax1.axhline(y=0.6**2, color='red', linestyle='--', label=f'理论值 ({0.6**2:.1%})')
ax1.set_xlabel('样本量')
ax1.set_ylabel('方差缩减')
ax1.set_title('样本量对方差缩减的影响')
ax1.legend()
ax1.set_xscale('log')

# 方差缩减的稳定性
ax2 = axes[1]
ax2.plot(sample_sizes, var_reduction_stds, 'o-', color='#e74c3c')
ax2.set_xlabel('样本量')
ax2.set_ylabel('方差缩减的标准差')
ax2.set_title('样本量对估计稳定性的影响')
ax2.axhline(y=0.05, color='gray', linestyle='--', alpha=0.5, label='5% 阈值')
ax2.set_xscale('log')
ax2.legend()

plt.tight_layout()
plt.show()

print("\n💡 关键发现：")
print("   样本量 < 200 时，方差缩减估计不稳定")
print("   建议：至少 200 样本/组才考虑使用 CUPED")
print("   样本量越大，CUPED 效果越接近理论值")

---

## Part 6: CUPED 最佳实践清单

### 🧪 TODO 练习: 实现 CUPED 前置检查

请实现一个完整的 CUPED 前置检查函数，在应用 CUPED 前验证各项条件。

In [ ]:
def cuped_preflight_check(Y_control, X_control, Y_treatment, X_treatment, 
                          min_correlation=0.3, min_sample_size=200, alpha=0.05):
    """
    CUPED 前置检查
    
    检查项：
    1. 样本量是否足够
    2. 相关性是否足够强
    3. 协变量是否有缺失值
    4. 协变量是否在处理前测量（通过检查组间平衡性）
    
    Returns:
        passed: 是否通过所有检查
        report: 详细检查报告
    """
    checks = []
    passed = True
    
    # 检查 1: 样本量
    n_c, n_t = len(Y_control), len(Y_treatment)
    sample_size_ok = n_c >= min_sample_size and n_t >= min_sample_size
    checks.append({
        'name': '样本量检查',
        'passed': sample_size_ok,
        'message': f"控制组: {n_c}, 实验组: {n_t} (最低要求: {min_sample_size})",
        'severity': 'error' if not sample_size_ok else 'ok'
    })
    if not sample_size_ok:
        passed = False
    
    # 检查 2: 相关性
    X_valid_c = X_control[~np.isnan(X_control)]
    Y_valid_c = Y_control[~np.isnan(X_control)]
    X_valid_t = X_treatment[~np.isnan(X_treatment)]
    Y_valid_t = Y_treatment[~np.isnan(X_treatment)]
    
    if len(X_valid_c) > 2 and len(X_valid_t) > 2:
        X_all = np.concatenate([X_valid_c, X_valid_t])
        Y_all = np.concatenate([Y_valid_c, Y_valid_t])
        rho, p_val = stats.pearsonr(X_all, Y_all)
        
        corr_ok = abs(rho) >= min_correlation
        checks.append({
            'name': '相关性检查',
            'passed': corr_ok,
            'message': f"ρ = {rho:.3f} (最低要求: {min_correlation}), 理论方差缩减: {rho**2:.1%}",
            'severity': 'warning' if not corr_ok else 'ok'
        })
    else:
        checks.append({
            'name': '相关性检查',
            'passed': False,
            'message': '有效数据不足，无法计算相关性',
            'severity': 'error'
        })
        passed = False
    
    # 检查 3: 缺失值
    missing_c = np.isnan(X_control).sum()
    missing_t = np.isnan(X_treatment).sum()
    missing_ratio = (missing_c + missing_t) / (n_c + n_t)
    
    missing_ok = missing_ratio < 0.3
    checks.append({
        'name': '缺失值检查',
        'passed': missing_ok,
        'message': f"控制组缺失: {missing_c}, 实验组缺失: {missing_t} ({missing_ratio:.1%})",
        'severity': 'warning' if missing_ratio > 0.1 else 'ok'
    })
    
    # 检查 4: 协变量分布平衡性（实验前应该平衡）
    if len(X_valid_c) > 2 and len(X_valid_t) > 2:
        t_stat, p_val = stats.ttest_ind(X_valid_c, X_valid_t)
        balance_ok = p_val > alpha
        checks.append({
            'name': '协变量平衡性检查',
            'passed': balance_ok,
            'message': f"p-value = {p_val:.4f}" + (" (不平衡，可能存在问题!)" if not balance_ok else " (平衡)"),
            'severity': 'warning' if not balance_ok else 'ok'
        })
    
    # 打印报告
    print("=" * 60)
    print("CUPED 前置检查报告")
    print("=" * 60)
    
    for check in checks:
        status = '✅' if check['passed'] else ('⚠️' if check['severity'] == 'warning' else '❌')
        print(f"\n{status} {check['name']}")
        print(f"   {check['message']}")
    
    print("\n" + "=" * 60)
    if passed:
        print("✅ 所有关键检查通过，可以使用 CUPED")
    else:
        print("❌ 存在关键问题，建议不使用 CUPED 或先解决问题")
    
    return passed, checks

print("✅ CUPED 前置检查函数已实现")

### 📝 参考答案

In [ ]:
# ============================================================
# 参考答案：CUPED 前置检查完整实现
# ============================================================

def cuped_preflight_check(Y_control, X_control, Y_treatment, X_treatment, 
                          min_correlation=0.3, min_sample_size=200, alpha=0.05):
    """
    CUPED 前置检查 - 完整实现
    """
    checks = []
    passed = True
    
    # 检查 1: 样本量
    n_c, n_t = len(Y_control), len(Y_treatment)
    sample_size_ok = n_c >= min_sample_size and n_t >= min_sample_size
    checks.append({
        'name': '样本量检查',
        'passed': sample_size_ok,
        'message': f"控制组: {n_c}, 实验组: {n_t} (最低要求: {min_sample_size})",
        'severity': 'error' if not sample_size_ok else 'ok'
    })
    if not sample_size_ok:
        passed = False
    
    # 检查 2: 相关性
    X_valid_c = X_control[~np.isnan(X_control)]
    Y_valid_c = Y_control[~np.isnan(X_control)]
    X_valid_t = X_treatment[~np.isnan(X_treatment)]
    Y_valid_t = Y_treatment[~np.isnan(X_treatment)]
    
    if len(X_valid_c) > 2 and len(X_valid_t) > 2:
        X_all = np.concatenate([X_valid_c, X_valid_t])
        Y_all = np.concatenate([Y_valid_c, Y_valid_t])
        rho, p_val = stats.pearsonr(X_all, Y_all)
        
        corr_ok = abs(rho) >= min_correlation
        checks.append({
            'name': '相关性检查',
            'passed': corr_ok,
            'message': f"ρ = {rho:.3f} (最低要求: {min_correlation}), 理论方差缩减: {rho**2:.1%}",
            'severity': 'warning' if not corr_ok else 'ok'
        })
    else:
        checks.append({
            'name': '相关性检查',
            'passed': False,
            'message': '有效数据不足，无法计算相关性',
            'severity': 'error'
        })
        passed = False
    
    # 检查 3: 缺失值
    missing_c = np.isnan(X_control).sum()
    missing_t = np.isnan(X_treatment).sum()
    missing_ratio = (missing_c + missing_t) / (n_c + n_t)
    
    missing_ok = missing_ratio < 0.3
    checks.append({
        'name': '缺失值检查',
        'passed': missing_ok,
        'message': f"控制组缺失: {missing_c}, 实验组缺失: {missing_t} ({missing_ratio:.1%})",
        'severity': 'warning' if missing_ratio > 0.1 else 'ok'
    })
    
    # 检查 4: 协变量分布平衡性
    if len(X_valid_c) > 2 and len(X_valid_t) > 2:
        t_stat, p_val = stats.ttest_ind(X_valid_c, X_valid_t)
        balance_ok = p_val > alpha
        checks.append({
            'name': '协变量平衡性检查',
            'passed': balance_ok,
            'message': f"p-value = {p_val:.4f}" + (" (不平衡，可能存在问题!)" if not balance_ok else " (平衡)"),
            'severity': 'warning' if not balance_ok else 'ok'
        })
    
    # 打印报告
    print("=" * 60)
    print("CUPED 前置检查报告")
    print("=" * 60)
    
    for check in checks:
        status = '✅' if check['passed'] else ('⚠️' if check['severity'] == 'warning' else '❌')
        print(f"\n{status} {check['name']}")
        print(f"   {check['message']}")
    
    print("\n" + "=" * 60)
    if passed:
        print("✅ 所有关键检查通过，可以使用 CUPED")
    else:
        print("❌ 存在关键问题，建议不使用 CUPED 或先解决问题")
    
    return passed, checks

# 测试
print("测试场景 1: 正常情况")
X_c = np.random.randn(500)
Y_c = 0.7 * X_c + np.random.randn(500) * 0.5
X_t = np.random.randn(500)
Y_t = 0.7 * X_t + np.random.randn(500) * 0.5 + 2

cuped_preflight_check(Y_c, X_c, Y_t, X_t)

print("\n" + "#" * 70 + "\n")

print("测试场景 2: 问题场景（样本量小、相关性低）")
X_c_small = np.random.randn(50)
Y_c_small = 0.1 * X_c_small + np.random.randn(50)
X_t_small = np.random.randn(50)
Y_t_small = 0.1 * X_t_small + np.random.randn(50) + 2

cuped_preflight_check(Y_c_small, X_c_small, Y_t_small, X_t_small)

---

## 💡 思考题

1. **协变量选择**：如果有多个候选协变量，如何选择最佳的？
   - 提示：考虑相关性、稳定性、业务意义

2. **多协变量 CUPED**：如果想同时使用多个协变量，该如何处理？
   - 提示：多元回归、CUPAC

3. **CUPED 的边界**：在什么情况下，CUPED 可能比原始方法更差？
   - 提示：小样本、估计误差

4. **与分层抽样的关系**：CUPED 和分层抽样都能减少方差，两者有什么区别？
   - 提示：事前 vs 事后，适用场景

---

## 📝 总结

### CUPED 失败模式总结

| 失败模式 | 原因 | 诊断方法 | 解决方案 |
|---------|------|---------|----------|
| 相关性低 | ρ < 0.3 | 计算相关系数 | 选择更好的协变量 |
| 新用户无数据 | 缺失值 | 检查缺失比例 | 分层 CUPED |
| 协变量受影响 | 实验期间数据 | 检查数据时间 | 使用实验前数据 |
| 样本量过小 | θ 估计不稳定 | 检查样本量 | 至少 200/组 |

### 最佳实践清单

- [ ] 使用**实验前**的数据作为协变量
- [ ] 检查相关性 ≥ 0.3
- [ ] 确保样本量 ≥ 200/组
- [ ] 处理缺失值（分层 CUPED）
- [ ] 验证协变量在组间平衡
- [ ] 报告方差缩减效果

### 面试要点

- 能解释 CUPED 的数学原理
- 能列举 CUPED 的适用条件
- 能诊断 CUPED 失效的原因
- 能设计 CUPED 的前置检查流程